In [ ]:
!python -V

Python 3.10.12


pip freeze>requirements.txt para descargar las versiones de las librerias

In [ ]:
pip install transformers[torch]==4.26.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1

In [ ]:
pip install accelerate==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 36.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip freeze>requirements.txt

In [ ]:
import pandas as pd
from datasets import Dataset

# Cargar datos de FAQs desde un CSV
df = pd.read_csv('/content/mental_health_faq.csv')
df = df.drop(columns=['Question_ID'])
# Convertir los datos en un formato adecuado
data = [{"text": f"Question: {row['Questions']} Answer: {row['Answers']}"} for _, row in df.iterrows()]

# Crear el dataset de Hugging Face
dataset = Dataset.from_list(data)

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from flask import Flask, request, jsonify
from transformers import pipeline
# Cargar el tokenizer y el modelo preentrenado
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Agregar token de padding al tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenizar los datos y agregar labels
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir='./logs',
)

# Entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

trainer.train()

# Guardar el modelo ajustado
model.save_pretrained('./faqs_gpt2')
tokenizer.save_pretrained('./faqs_gpt2')

# Configurar un Chatbot con Flask
app = Flask(__name__)

# Cargar el modelo y el tokenizer ajustados
model = GPT2LMHeadModel.from_pretrained('./faqs_gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./faqs_gpt2')

# Crear un pipeline de generación de texto
faq_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    prompt = f"Question: {user_input} Answer:"
    response = faq_pipeline(prompt, max_length=150, num_return_sequences=1)
    answer = response[0]['generated_text'].replace(prompt, "").strip()
    return jsonify({'response': answer})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 98
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 147
  Number of trainable parameters = 124439808


Epoch,Training Loss,Validation Loss
1,No log,1.499543
2,No log,1.413434
3,No log,1.393323


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 2


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved 

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from google.colab import drive
import os
import shutil

# Montar Google Drive
drive.mount('/content/drive')

# Verificar la existencia de la carpeta
folder_path = '/content/faqs_gpt2'
if os.path.exists(folder_path):
    print("La carpeta existe.")
else:
    print("La carpeta no existe.")
    # Terminar el proceso si la carpeta no existe
    exit()

# Especificar la ruta en Google Drive
destination_path = '/content/drive/My Drive/faqs_gpt2'

# Copiar la carpeta a Google Drive
shutil.copytree(folder_path, destination_path)

# Verificar que la carpeta se ha copiado correctamente
if os.path.exists(destination_path):
    print(f'Carpeta guardada en: {destination_path}')
else:
    print("Error al guardar la carpeta.")

Mounted at /content/drive
La carpeta existe.
Carpeta guardada en: /content/drive/My Drive/faqs_gpt2


In [ ]:
# Probar el modelo directamente con inputs
# Cargar el modelo y el tokenizer ajustados
model = GPT2LMHeadModel.from_pretrained('./faqs_gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./faqs_gpt2')

# Crear un pipeline de generación de texto
faq_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Función para interactuar con el chatbot
def chat_with_bot(question):
    prompt = f"Question: {question} Answer:"
    response = faq_pipeline(prompt, max_length=150, num_return_sequences=1)
    answer = response[0]['generated_text'].replace(prompt, "").strip()
    return answer

# Ejemplo de uso
if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            break
        bot_response = chat_with_bot(user_input)
        print(f"Bot: {bot_response}")

loading configuration file ./faqs_gpt2/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights f

You: hola


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: hola is a name for an aggressive, cold or flu infection caused by the virus. It is a cold, flu-like illness. People who are ill may experience symptoms or reactions such as headaches, muscle aches, tiredness, changes in appetite, or even a fear of someone who may have colds or flu, says Dr. Paul Schaffer, M.D., Ph.D, of the Infectious Disease Laboratory at Stanford University Medical Center.
You: How i resolve anxiety?


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: Anxiety is a natural response. If you have anxiety, chances are you know how to cope. However, it is better to talk this out with your doctor rather than your therapist. Here is what you need to know. Anxiety and depression are very different experiences, and there are a lot of options. Your doctor will tell you what's known for some reasons. If you have a family history of anxiety, you may have a mental health disorder that can make symptoms worse. When anxiety is diagnosed, you have the chance to experience life changing changes. Anxiety is a complex symptom that changes over time and can affect you during different parts of life. Anxiety is related to problems in school, work, social life and


KeyboardInterrupt: Interrupted by user